In [2]:
import rasterio
from rasterio.enums import Resampling
import numpy as np
import os

# Define the base directory containing images
base_dir = "/Users/wasifqazi/Desktop/FYP_Sentinel_2/Sentinel_2_Data/scenes_data"

# Define the output directory where stacked images will be saved
output_dir = "/Users/wasifqazi/Desktop/FYP_Sentinel_2/output_path"

# Ensure the output directory exists
os.makedirs(output_dir, exist_ok=True)

# Define required bands
required_bands = {
    "B02": "10m",  # Blue
    "B03": "10m",  # Green
    "B04": "10m",  # Red
    "B08": "10m",  # NIR
    "B05": "20m",  # Red Edge 1
    "B06": "20m",  # Red Edge 2
    "B07": "20m",  # Red Edge 3
    "B8A": "20m",  # NIR Narrow
    "B11": "20m",  # SWIR 1
    "B12": "20m",  # SWIR 2
    "B01": "60m",  # Coastal Aerosol
    "B09": "60m",  # Water Vapor
}

# Loop through each image folder
for image_folder in os.listdir(base_dir):
    folder_path = os.path.join(base_dir, image_folder)
    if not os.path.isdir(folder_path):
        continue  # Skip non-directory files

    print(f"Processing folder: {image_folder}")

    # Gather paths for required bands only
    band_paths = {}
    for filename in os.listdir(folder_path):
        for band_name, resolution in required_bands.items():
            # Check if the band name and resolution both appear in the filename
            if band_name in filename and resolution in filename and filename.endswith(".jp2"):
                band_paths[band_name] = os.path.join(folder_path, filename)

    # Check if all required bands are present
    if len(band_paths) != len(required_bands):
        print(f"Skipping {image_folder}: Missing required bands.")
        continue

    # Sort band paths by the order in required_bands
    sorted_band_paths = [band_paths[band] for band in required_bands]

    # Use the first 10m band as the reference for resolution and metadata
    reference_band = next(
        (path for band, path in band_paths.items() if required_bands[band] == "10m"), None
    )
    if reference_band is None:
        print(f"Skipping {image_folder}: No 10m reference band found.")
        continue

    with rasterio.open(reference_band) as ref:
        ref_transform = ref.transform
        ref_crs = ref.crs
        ref_width = ref.width
        ref_height = ref.height

    # Stack the required bands (with resampling if necessary)
    stacked_bands = []
    for band, band_path in band_paths.items():
        with rasterio.open(band_path) as src:
            # Resample to match reference resolution (10m, in this case)
            if src.width != ref_width or src.height != ref_height:
                print(f"Resampling {os.path.basename(band_path)} to 10m resolution...")
                data = src.read(
                    out_shape=(1, ref_height, ref_width),
                    resampling=Resampling.bilinear,
                )[0]
            else:
                data = src.read(1)
            
            stacked_bands.append(data)

    # Validate band shapes (ensure all bands have the same shape)
    shapes = [band.shape for band in stacked_bands]
    if len(set(shapes)) > 1:
        print(f"Skipping {image_folder}: Bands have inconsistent shapes: {shapes}")
        continue

    # Stack the bands
    stacked_array = np.stack(stacked_bands, axis=0)

    # Write the stacked raster for the current image
    output_path = os.path.join(output_dir, f"{image_folder}_stacked.jp2")
    with rasterio.open(
        output_path,
        "w",
        driver="JP2OpenJPEG",
        height=ref_height,
        width=ref_width,
        count=len(stacked_bands),
        dtype=stacked_array.dtype,
        crs=ref_crs,
        transform=ref_transform,
    ) as dst:
        for i in range(stacked_array.shape[0]):
            dst.write(stacked_array[i, :, :], i + 1)

    print(f"Stacked image saved to: {output_path}") 

Processing folder: Image_32
Resampling T29UPT_20241222T113409_B12_20m.jp2 to 10m resolution...
Resampling T29UPT_20241222T113409_B09_60m.jp2 to 10m resolution...
Resampling T29UPT_20241222T113409_B11_20m.jp2 to 10m resolution...
Resampling T29UPT_20241222T113409_B01_60m.jp2 to 10m resolution...
Resampling T29UPT_20241222T113409_B05_20m.jp2 to 10m resolution...
Resampling T29UPT_20241222T113409_B06_20m.jp2 to 10m resolution...
Resampling T29UPT_20241222T113409_B8A_20m.jp2 to 10m resolution...
Resampling T29UPT_20241222T113409_B07_20m.jp2 to 10m resolution...
Stacked image saved to: /Users/wasifqazi/Desktop/FYP_Sentinel_2/output_path/Image_32_stacked.jp2
Processing folder: Image_3
Resampling T29UPV_20250131T113319_B05_20m.jp2 to 10m resolution...
Resampling T29UPV_20250131T113319_B06_20m.jp2 to 10m resolution...
Resampling T29UPV_20250131T113319_B8A_20m.jp2 to 10m resolution...
Resampling T29UPV_20250131T113319_B07_20m.jp2 to 10m resolution...
Resampling T29UPV_20250131T113319_B12_20m.jp